# 1. Setup

In [1]:
import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import set_seed
import torch
from typing import Callable
import random
import os
# import langchain
# from langchain_community.llms import HuggingFaceHub
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_community.embeddings import HuggingFaceHubEmbeddings
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.docstore.document import Document
# from langchain.vectorstores import FAISS
# from langchain_community.vectorstores.utils import DistanceStrategy

/w5home/lxu3/anaconda3/envs/kmGPT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Configuration variables for this whole notebook
class config:
    device = "cuda"
    seed = 42
    model = "Open-Orca/Mistral-7B-OpenOrca"
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
    top_k = 20
    top_p = 0.95
    do_sample = True
    num_return_sequences = 1
    max_new_tokens = 500
    temperature = 0.8
    repetition_penalty = 1.2
    penalty_alpha=0.6


In [9]:
# From this Gist: https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    set_seed(seed)

In [10]:
seed_everything(config.seed)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(config.model, quantization_config = config.bnb_config, torch_dtype = torch.bfloat16, device_map = config.device, trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained(config.model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
data = pd.read_csv("/content/data.csv")
data

# 1. Use Mistral7B/Open Orca to do Hypothesis Evaluation (First Idea)

> **Justification for this model (if curious):**
1. Mistral 7B is one of the best models out there
2. Orca has shown to do really well in imitating GPT-4's responses
3. Together, they've been shown to have 98.6% of Llama2-70b-chat's performance!

References:
https://arxiv.org/pdf/2306.02707.pdf
https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca
https://arxiv.org/abs/2310.06825

**Accuracy:** 33%

In [ ]:
hyp = "ezetimibe may effectively alleviate or target key pathogenic mechanisms of diabetes potentially offering therapeutic benefits or slowing disease progression."

In [ ]:
def retrieveZeroShotPrompt(abstract: str) -> str:
  zero_shot_prompt = f"""
    Hypothesis: {hyp}
    Abstract: {abstract}

    Determine whether or not this abstract is relevant for scientifically evaluating the provided hypothesis.
    A relevant abstract should either support the given hypothesis or have evidence to refute the hypothesis.
    A relevant abstract must directly comment on the hypothesis.

    Give your answer as either 0 (Not Relevant) or 1 (Relevant).
    Answer:
  """
  return zero_shot_prompt

In [ ]:
# Sections from Mistral-7B-OpenOrca Notebook
# https://colab.research.google.com/drive/1yZlLSifCGELAX5GN582kZypHCv0uJuNX?usp=sharing#scrollTo=C65lxy1PBr5m
def generateInput(abstract: str, retrievePrompt: Callable) -> str:
  prompt = retrievePrompt(abstract)
  sys_prompt = "You are an incredibly brilliant biomedical researcher who has spent their lifetime reading all the papers in PubMed. You are focused on assisting other researchers in evaluating suggested hypotheses given abstracts in PubMed."
  prefix = "<|im_start|>"
  suffix = "<|im_end|>\n"

  sys_format = prefix + "system\n" + sys_prompt + suffix
  user_format = prefix + "user\n" + prompt + suffix
  assistant_format = prefix + "assistant\n"
  input_text = sys_format + user_format + assistant_format

  return input_text

In [ ]:
def generateOutput(retievePrompt: Callable) -> list:
  abstracts = data["abstract"]
  prompts = [generateInput(abstract, retievePrompt) for abstract in abstracts]
  inputs = tokenizer(prompts, return_tensors = "pt", padding = True)
  output = model.generate(**inputs, do_sample = True, temperature = config.temperature,
                          top_k = config.top_k, top_p = config.top_p,
                          num_return_sequences = config.num_return_sequences,
                          max_new_tokens = config.max_new_tokens,
                          repetition_penalty = config.repetition_penalty,
                          penalty_alpha = config.penalty_alpha)
  processed_outputs = output[:, inputs["input_ids"].shape[1]:] # Removes input prompt from output

  return processed_outputs

In [ ]:
def printOutputs(outputs: list):
  for response in outputs:
    print(tokenizer.decode(response))
    print("--------------------------")

# 1.5 CoT Prompt

> **Justification:**
1. CoT can increase performance

**Accuracy:** 83%

**Note:** The model repeats end tokens a lot, but adding few-shot examples along with CoT should solve the issue.

**Improvements:** Validate model outputs with Pydantic

References:
https://arxiv.org/abs/2201.11903


In [ ]:
def retrieveZeroShotCoTPrompt(abstract: str) -> str:
  zero_shot_prompt = f"""
    Hypothesis: {hyp}
    Abstract: {abstract}

    Determine whether or not this abstract is relevant for scientifically evaluating the provided hypothesis.
    A relevant abstract should either support the given hypothesis or have evidence to refute the hypothesis.
    A relevant abstract must directly comment on the hypothesis.

    Give your answer as either 0 (Not Relevant) or 1 (Relevant).
    Answer: Let us think through this step by step.
  """
  return zero_shot_prompt

In [ ]:
%%time
processed_zero_shot_CoT_outputs = generateOutput(retrieveZeroShotCoTPrompt)

In [ ]:
processed_zero_shot_CoT_outputs

In [ ]:
printOutputs(processed_zero_shot_CoT_outputs)

In [ ]:
data.iloc[-1]["abstract"]

In [ ]:
answers = np.array([0, 0, 0, 1, 1, 1])
zero_shot_CoT_acc = (np.array(data["label"]) == answers).sum() / len(answers)
zero_shot_CoT_acc